# PtyPy QC & Ingestion — **Demo Variant**
This notebook demonstrates a minimal, **repo-driven** workflow:
1. **Optionally** generate a *small synthetic dataset* (so the demo runs anywhere).
2. Ingest to a standardized HDF5 using the repo’s CLI.
3. (Optionally) choose an ROI interactively.
4. Run the **repo evaluator** to produce **Outliers**, **STXM**, **DPC‑X**, and **DPC‑Y** maps.
5. Preview outputs.

**Note:** This variant deliberately **does not include any in-notebook fallback evaluator**. It is a demonstration of how to use your workflow as shipped in the repo.


## Prerequisites
- Local clone of `beamline_ptypy_Ingest` (or set `PTYPY_INGEST_REPO`).
- The package/CLIs importable (e.g., `python -m beamline_ptypy_ingest.ingest_hdf5 --help`).
- Evaluator at `ingest/evaluate.py`.


In [ ]:
# ---- Config ----
from pathlib import Path
import os, sys

# Repo location
REPO_ROOT = Path(os.environ.get("PTYPY_INGEST_REPO", ".")).resolve()
if (REPO_ROOT / "beamline_ptypy_ingest").exists():
    sys.path.insert(0, str(REPO_ROOT))

# Demo flags
GENERATE_SYNTHETIC = True   # <- set False to use your real RAW_HDF5 below
INTERACTIVE_SCAN_ROI    = False   # <- set False to skip the click-based ROI selection  [CURRENTLY BROKEN!]

# If not generating synthetic data, point RAW_HDF5 at your source file:
RAW_HDF5 = Path(os.environ.get("RAW_HDF5", REPO_ROOT / "tests" / "example_input.h5"))

# Outputs
OUT_DIR      = REPO_ROOT / "qc_outputs_demo"
STD_HDF5     = OUT_DIR / "standardized_demo.h5"
QC_DIR       = OUT_DIR / "qc_run"
OUT_DIR.mkdir(parents=True, exist_ok=True)
QC_DIR.mkdir(parents=True, exist_ok=True)

print("REPO_ROOT =", REPO_ROOT)
print("OUT_DIR   =", OUT_DIR)

# Sanity: required tools
def _require_repo_tools():
    import importlib
    try:
        importlib.import_module('ingest.ingest_hdf5')
    except Exception as e:
        raise RuntimeError("Cannot import 'ingest.ingest_hdf5'. "
                           "Ensure the repo package is on PYTHONPATH or installed.") from e
    eval_py = REPO_ROOT / "ingest" / "evaluate.py"
    if not eval_py.exists():
        raise FileNotFoundError(f"Evaluator not found at {eval_py}")
    return eval_py

EVAL_PY = _require_repo_tools()
print("Evaluator:", EVAL_PY)

In [ ]:
# ---- (Optional) Generate a tiny synthetic dataset ----
import numpy as np, h5py
from ingest.evaluate import make_synthetic_cxi

if GENERATE_SYNTHETIC:
    RAW_HDF5 = OUT_DIR / "raw_synthetic.h5"
    RAW_HDF5.parent.mkdir(parents=True, exist_ok=True) 
    _ = make_synthetic_cxi(RAW_HDF5)
    print("Synthetic dataset created at:", RAW_HDF5)
else:
    print("Using existing RAW_HDF5:", RAW_HDF5)

In [ ]:
# ---- Ingest to standardized HDF5 (via  CLI) ----
import subprocess, shlex, sys

ROI_NY, ROI_NX = 256, 256    # default ROI size
ROI_CY, ROI_CX = 0.5, 0.5    # default normalized centre, fraction of image size

cmd = [
    sys.executable, "-m", "ingest.ingest_hdf5",
    "--input", str(RAW_HDF5),
    "--output", str(STD_HDF5),
    "--frames-per-pos", "1",
    "--grouping", "none",
    "--roi", f"{ROI_NY},{ROI_NX}",
    "--roi-center", f"{ROI_CY},{ROI_CX}",
    "--rotate", "0",
    "--flip", "none",
    "--sat", "65000"
]
print("Running:\n", " ".join(shlex.quote(c) for c in cmd))
subprocess.run(cmd, check=True)
print("Standardised file:", STD_HDF5)

# quick check of scan dimensions in standardised file
from ingest.evaluate import read_positions
with h5py.File(STD_HDF5, "r") as h5:
    pos_x, pos_y = read_positions(h5, posx_path='/entry/pos', posy_path='/entry/pos')

print(f"\nScan positions shape: {len(pos_x)} x {len(pos_y)} (N={len(pos_x)*len(pos_y)})")
print("x range (µm):", 1.e6*pos_x.min(), "→", 1.e6*pos_x.max())
print("y range (µm):", 1.e6*pos_y.min(), "→", 1.e6*pos_y.max())

import matplotlib.pyplot as plt
plt.scatter(pos_x, pos_y, s=5); 
plt.gca().set_aspect('equal')
plt.title("Scan positions (x,y)"); plt.xlabel("x (m)"); plt.ylabel("y (m)")
plt.show()  # show scan positions


In [ ]:
# ---- (Optional) Re-ingest data with reduced ROI in diffraction patterns ----

import h5py, numpy as np, matplotlib.pyplot as plt, subprocess, shlex, sys


ROI_NY, ROI_NX = 64, 64    # reduced diffraction pattern ROI size
ROI_CY, ROI_CX = 0.5, 0.5    # default normalized centre, fraction of image size

# re-run ingest to apply chosen ROI 
cmd = [
    sys.executable, "-m", "ingest.ingest_hdf5",
    "--input", str(RAW_HDF5),
    "--output", str(STD_HDF5),
    "--frames-per-pos", "1",
    "--grouping", "none",
    "--roi", f"{ROI_NY},{ROI_NX}",
    "--roi-center", f"{ROI_CY:.6f},{ROI_CX:.6f}",
    "--rotate", "0",
    "--flip", "none",
    "--sat", "65000"
]
print("Re-running ingest with chosen ROI centre:\n".format(shlex.quote(c) for c in cmd))
subprocess.run(cmd, check=True)


In [ ]:
# ---- Run the evaluator ----
import subprocess, shlex, sys

cmd = [
    sys.executable, str(EVAL_PY), str(STD_HDF5),
    "--data-path", "/entry/data",
    "--pos-x-path", "/entry/pos",
    "--pos-y-path", "/entry/pos",
    "--roi", "256",
    "--bin", "1",
    "--embed-side", "32",
    "--pca-k", "12",
    "--w-feat", "0.5",
    "--reject-topp", "0.02",
    "--emit-ptypy-json", str(QC_DIR / "ptypy_indices.json"),
    "--thumbs-topk", "24",
    "--thumbs-size", "256",
    "--out", str(QC_DIR)
]


print("Running evaluator:\n".format(shlex.quote(c) for c in cmd))
subprocess.run(cmd, check=True)
print("QC outputs written to:", QC_DIR)

In [ ]:
# ---- Preview results ----
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

panel = QC_DIR / "qc_maps.png"
if panel.exists():
    img = mpimg.imread(panel)
    plt.figure(figsize=(6,6))
    plt.imshow(img); plt.axis('off')
    #plt.title("QC panel: Outliers / STXM / DPC-Y / DPC-X")
    plt.show()
else:
    print("qc_maps.png not found in", QC_DIR)